In [1]:
#-*-coding:utf-8-*-
import util
import tensorflow as tf
MNIST_CLASSIFIER_FROZEN_GRAPH = './classify_mnist_graph_def.pb'
INPUT_TENSOR = 'inputs:0'
OUTPUT_TENSOR = 'logits:0'
def EuclideanDistances(A, B):

    BT = B.transpose()
    # vecProd = A * BT
    vecProd = np.dot(A, BT)
    # print(vecProd)
    SqA = A ** 2
    # print(SqA)
    sumSqA = np.matrix(np.sum(SqA, axis=1))
    sumSqAEx = np.tile(sumSqA.transpose(), (1, vecProd.shape[1]))
    # print(sumSqAEx)

    SqB = B ** 2
    sumSqB = np.sum(SqB, axis=1)
    sumSqBEx = np.tile(sumSqB, (vecProd.shape[0], 1))
    SqED = sumSqBEx + sumSqAEx - 2 * vecProd
    SqED[SqED < 0] = 0.0
    ED = np.sqrt(SqED)
    return np.divide(ED.sum(),ED.shape[0]*ED.shape[1])
def cal_distance_image_real(images,labels):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)
def cal_distance_image_fake(images):
    eval_images=tf.convert_to_tensor(images)
    y_logits=util.mnist_logits(eval_images, MNIST_CLASSIFIER_FROZEN_GRAPH, INPUT_TENSOR, OUTPUT_TENSOR)
    y_logits=tf.Session().run(y_logits)
    labels = tf.Session().run(tf.argmax(y_logits, 1))
    dict={}
    all_dis=[]
    for i in range(10):
        dict[i]=[]
    for i in range(len(labels)):
        dict[labels[i]].append(y_logits[i])
    for i in range(10):
        dict[i]=np.array(dict[i])
        if len(dict[i]):
            dis = EuclideanDistances(dict[i], dict[i])  # 生成图片的紧度
        else:
            dis = -1
        all_dis.append(dis)
    return np.array(all_dis)

    

In [2]:
import os
if not os.path.isdir('saved_models_{}'.format('sgan')):
    os.mkdir('saved_models_{}'.format('sgan'))
f = open('saved_models_{}/log_collapse1.txt'.format('sgan'), mode='w')
import cv2
import torch.utils.data as Data

from keras.datasets import mnist,fashion_mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt
import os
import numpy as np

class SGAN:
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100
        self.x = []
        self.y = np.zeros((31, 1), dtype=np.int)
        self.y = list(self.y)
        for i in range(31):
            self.y[i] = []

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss=['binary_crossentropy', 'categorical_crossentropy'],
            loss_weights=[0.5, 0.5],
            optimizer=optimizer,
            metrics=['accuracy']
        )

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        noise = Input(shape=(100,))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid, _ = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(noise, valid)
        self.combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(1, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())

        model.summary()

        img = Input(shape=self.img_shape)

        features = model(img)
        valid = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [valid, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        X_test = (X_test.astype(np.float32) - 127.5) / 127.5
        X_test = np.expand_dims(X_test, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Class weights:
        # To balance the difference in occurences of digit class labels.
        # 50% of labels that the discriminator trains on are 'fake'.
        # Weight = 1 / frequency
        half_batch = batch_size // 2
        cw1 = {0: 1, 1: 1}
        cw2 = {i: self.num_classes / half_batch for i in range(self.num_classes)}
        cw2[self.num_classes] = 1 / half_batch

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                labels = to_categorical(y_train[index * batch_size:(index + 1) * batch_size], num_classes=self.num_classes + 1)
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # One-hot encoding of labels
                # labels = to_categorical(y_train[idx], num_classes=self.num_classes + 1)
                fake_labels = to_categorical(np.full((batch_size, 1), self.num_classes),
                                             num_classes=self.num_classes + 1)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                    epoch, global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss))

                sampleSize = 5000
                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    s = self.sample_images(global_step, X_test,y_test, sampleSize)
    def sample_images(self, epoch,x_test,y_test,sample_num):
        r, c = sample_num//10, 10
        noise = np.random.normal(0, 1, (r * c, 100))
#         sampled_labels = np.array([num for _ in range(r) for num in range(c)])
        gen_imgs = self.generator.predict([noise])
        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5
        labels = np.squeeze(y_test[:sample_num])
        labels = np.squeeze(labels)
        dis_real = cal_distance_image_real(x_test[:sample_num], labels)
        dis_fake = cal_distance_image_fake(gen_imgs)
        dis_cha = dis_real - dis_fake
        print('##############')
        # print(dis_real)
        # print(dis_fake)
        print(dis_cha)
        print('##########')
        f.writelines('\n')
        f.writelines('epoch:' + str(epoch))
        f.writelines('\n')
        f.writelines('紧度')
        f.writelines('\n')
        f.writelines(' %.8f ' % (i) for i in dis_cha)
        f.writelines('\n')

if __name__ == '__main__':
    sgan = SGAN()
    sgan.train(epochs=40, batch_size=64, sample_interval=200)



Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1[D loss: 0.696796, acc: 34.38%, op_acc: 10.94%] [G loss: 0.575394]
epoch:0 step:2[D loss: 0.562617, acc: 62.50%, op_acc: 15.62%] [G loss: 0.883754]
epoch:0 step:3[D loss: 0.509350, acc: 70.31%, op_acc: 10.94%] [G loss: 0.942662]
epoch:0 step:4[D loss: 0.443015, acc: 76.56%, op_acc: 11.72%] [G loss: 1.080559]
epoch:0 step:5[D loss: 0.427373, acc: 75.78%, op_acc: 19.53%] [G loss: 1.232979]
epoch:0 step:6[D loss: 0.390650, acc: 84.38%, op_acc: 21.88%] [G loss: 1.250112]
epoch:0 step:7[D loss: 0.434201, acc: 77.34%, op_acc: 15.62%] [G loss: 1.303827]
epoch:0 step:8[D loss: 0.405340, acc: 79.69%, op_acc: 21.09%] [G loss: 1.376235]
epoch:0 step:9[D loss: 0.404122, acc: 80.47%, op_acc: 18.75%] [G loss: 1.489203]
epoch:0 step:10[D loss: 0.383595, acc: 82.03%, op_acc: 21.09%] [G loss: 1.509490]
epoch:0 step:11[D loss: 0.392738, acc: 78.91%, op_acc: 25.78%] [G loss: 1.686248]
epoch:0 step:12[D loss: 0.364382, acc: 86.72%, op_acc: 25.78%] [G loss: 1.754028]
epoch:0 step:13[D loss: 0

epoch:0 step:103[D loss: 0.465817, acc: 50.78%, op_acc: 50.78%] [G loss: 2.198162]
epoch:0 step:104[D loss: 0.369754, acc: 71.09%, op_acc: 54.69%] [G loss: 1.678466]
epoch:0 step:105[D loss: 0.398005, acc: 66.41%, op_acc: 53.12%] [G loss: 1.465505]
epoch:0 step:106[D loss: 0.311784, acc: 75.78%, op_acc: 57.03%] [G loss: 1.507637]
epoch:0 step:107[D loss: 0.340536, acc: 75.78%, op_acc: 51.56%] [G loss: 1.542544]
epoch:0 step:108[D loss: 0.422502, acc: 63.28%, op_acc: 49.22%] [G loss: 1.467527]
epoch:0 step:109[D loss: 0.474002, acc: 51.56%, op_acc: 49.22%] [G loss: 1.849702]
epoch:0 step:110[D loss: 0.406499, acc: 68.75%, op_acc: 51.56%] [G loss: 2.003188]
epoch:0 step:111[D loss: 0.515495, acc: 49.22%, op_acc: 44.53%] [G loss: 1.648014]
epoch:0 step:112[D loss: 0.415566, acc: 60.94%, op_acc: 49.22%] [G loss: 1.698502]
epoch:0 step:113[D loss: 0.363746, acc: 67.19%, op_acc: 46.88%] [G loss: 1.665639]
epoch:0 step:114[D loss: 0.463471, acc: 57.81%, op_acc: 45.31%] [G loss: 1.405082]
epoc

...

epoch:39 step:37015[D loss: 0.415172, acc: 57.03%, op_acc: 47.66%] [G loss: 1.226976]
epoch:39 step:37016[D loss: 0.297852, acc: 77.34%, op_acc: 57.81%] [G loss: 1.175161]
epoch:39 step:37017[D loss: 0.262880, acc: 80.47%, op_acc: 59.38%] [G loss: 1.556105]
epoch:39 step:37018[D loss: 0.269341, acc: 78.91%, op_acc: 54.69%] [G loss: 1.758120]
epoch:39 step:37019[D loss: 0.330745, acc: 66.41%, op_acc: 57.03%] [G loss: 1.283719]
epoch:39 step:37020[D loss: 0.310165, acc: 72.66%, op_acc: 51.56%] [G loss: 1.377124]
epoch:39 step:37021[D loss: 0.303030, acc: 74.22%, op_acc: 53.91%] [G loss: 1.339169]
epoch:39 step:37022[D loss: 0.294393, acc: 75.78%, op_acc: 60.94%] [G loss: 1.499144]
epoch:39 step:37023[D loss: 0.417697, acc: 58.59%, op_acc: 53.91%] [G loss: 1.414563]
epoch:39 step:37024[D loss: 0.392240, acc: 57.81%, op_acc: 52.34%] [G loss: 1.203689]
epoch:39 step:37025[D loss: 0.334902, acc: 67.97%, op_acc: 53.12%] [G loss: 1.570865]
epoch:39 step:37026[D loss: 0.261389, acc: 80.47%, op_

epoch:39 step:37114[D loss: 0.415989, acc: 56.25%, op_acc: 47.66%] [G loss: 0.930464]
epoch:39 step:37115[D loss: 0.259931, acc: 82.03%, op_acc: 57.03%] [G loss: 1.268135]
epoch:39 step:37116[D loss: 0.422207, acc: 55.47%, op_acc: 46.88%] [G loss: 1.136005]
epoch:39 step:37117[D loss: 0.314290, acc: 71.88%, op_acc: 60.16%] [G loss: 1.255679]
epoch:39 step:37118[D loss: 0.298222, acc: 70.31%, op_acc: 55.47%] [G loss: 1.359145]
epoch:39 step:37119[D loss: 0.252603, acc: 80.47%, op_acc: 58.59%] [G loss: 1.709327]
epoch:39 step:37120[D loss: 0.368682, acc: 64.06%, op_acc: 58.59%] [G loss: 1.327986]
epoch:39 step:37121[D loss: 0.347215, acc: 71.88%, op_acc: 53.12%] [G loss: 1.480654]
epoch:39 step:37122[D loss: 0.324897, acc: 71.09%, op_acc: 54.69%] [G loss: 1.519547]
epoch:39 step:37123[D loss: 0.354427, acc: 70.31%, op_acc: 46.88%] [G loss: 1.570225]
epoch:39 step:37124[D loss: 0.362018, acc: 67.97%, op_acc: 49.22%] [G loss: 1.288321]
epoch:39 step:37125[D loss: 0.301986, acc: 74.22%, op_

epoch:39 step:37210[D loss: 0.330754, acc: 67.97%, op_acc: 50.00%] [G loss: 1.090350]
epoch:39 step:37211[D loss: 0.324000, acc: 73.44%, op_acc: 57.81%] [G loss: 1.525895]
epoch:39 step:37212[D loss: 0.345158, acc: 67.97%, op_acc: 50.78%] [G loss: 1.381789]
epoch:39 step:37213[D loss: 0.335032, acc: 70.31%, op_acc: 49.22%] [G loss: 1.226183]
epoch:39 step:37214[D loss: 0.291299, acc: 74.22%, op_acc: 62.50%] [G loss: 1.452196]
epoch:39 step:37215[D loss: 0.297765, acc: 76.56%, op_acc: 57.03%] [G loss: 1.234397]
epoch:39 step:37216[D loss: 0.347635, acc: 67.19%, op_acc: 57.81%] [G loss: 1.345682]
epoch:39 step:37217[D loss: 0.370757, acc: 70.31%, op_acc: 50.00%] [G loss: 1.434039]
epoch:39 step:37218[D loss: 0.346658, acc: 64.84%, op_acc: 53.12%] [G loss: 1.128899]
epoch:39 step:37219[D loss: 0.296799, acc: 74.22%, op_acc: 56.25%] [G loss: 1.464092]
epoch:39 step:37220[D loss: 0.351167, acc: 69.53%, op_acc: 50.78%] [G loss: 1.585800]
epoch:39 step:37221[D loss: 0.414383, acc: 60.94%, op_

epoch:39 step:37307[D loss: 0.297746, acc: 78.91%, op_acc: 57.03%] [G loss: 1.341280]
epoch:39 step:37308[D loss: 0.377387, acc: 64.84%, op_acc: 52.34%] [G loss: 1.163982]
epoch:39 step:37309[D loss: 0.324589, acc: 66.41%, op_acc: 56.25%] [G loss: 1.628734]
epoch:39 step:37310[D loss: 0.355328, acc: 64.84%, op_acc: 53.12%] [G loss: 1.563300]
epoch:39 step:37311[D loss: 0.381407, acc: 60.16%, op_acc: 54.69%] [G loss: 1.532716]
epoch:39 step:37312[D loss: 0.358554, acc: 60.94%, op_acc: 55.47%] [G loss: 1.071849]
epoch:39 step:37313[D loss: 0.373540, acc: 61.72%, op_acc: 48.44%] [G loss: 1.134977]
epoch:39 step:37314[D loss: 0.406432, acc: 58.59%, op_acc: 53.12%] [G loss: 1.581202]
epoch:39 step:37315[D loss: 0.267901, acc: 78.91%, op_acc: 61.72%] [G loss: 1.403128]
epoch:39 step:37316[D loss: 0.359870, acc: 64.84%, op_acc: 50.78%] [G loss: 1.399620]
epoch:39 step:37317[D loss: 0.428850, acc: 59.38%, op_acc: 50.78%] [G loss: 1.599500]
epoch:39 step:37318[D loss: 0.360703, acc: 64.84%, op_

epoch:39 step:37404[D loss: 0.261416, acc: 81.25%, op_acc: 57.03%] [G loss: 1.652935]
epoch:39 step:37405[D loss: 0.292726, acc: 72.66%, op_acc: 58.59%] [G loss: 1.589506]
epoch:39 step:37406[D loss: 0.246180, acc: 85.16%, op_acc: 61.72%] [G loss: 1.726549]
epoch:39 step:37407[D loss: 0.303993, acc: 70.31%, op_acc: 59.38%] [G loss: 1.378215]
epoch:39 step:37408[D loss: 0.350561, acc: 66.41%, op_acc: 52.34%] [G loss: 1.098774]
epoch:39 step:37409[D loss: 0.333297, acc: 67.97%, op_acc: 53.91%] [G loss: 1.518626]
epoch:39 step:37410[D loss: 0.261791, acc: 81.25%, op_acc: 57.03%] [G loss: 1.240750]
epoch:39 step:37411[D loss: 0.340582, acc: 68.75%, op_acc: 54.69%] [G loss: 1.207906]
epoch:39 step:37412[D loss: 0.257161, acc: 83.59%, op_acc: 60.94%] [G loss: 1.513879]
epoch:39 step:37413[D loss: 0.241983, acc: 82.03%, op_acc: 60.94%] [G loss: 1.764869]
epoch:39 step:37414[D loss: 0.405744, acc: 56.25%, op_acc: 48.44%] [G loss: 1.251164]
epoch:39 step:37415[D loss: 0.312434, acc: 71.09%, op_